## This script processes job vacancies data, performing initial cleaning tasks to prepare for in-depth analysis across various metrics such as job demand, geographic distribution, required skills, and industry trends.

**Step 1 : Importing modules**

In [1]:
import pandas as pd
from pathlib import Path
import re
from pandasql import sqldf

**Step 2 : Reading file from directory**

In [2]:
job_vacancies = Path("C:/Users/Shuvam/Desktop/Learnings/Data/jobs_data/jobs_data.csv")
vacancies = pd.read_csv(job_vacancies.resolve(), sep=',')

In [3]:
vacancies.head(3)

,Job Name,Company Name,JD,Skills,Date Posted,YOE,Location,Website,Job Function:,Industry:,Specialization:,Qualification:,Hiring Location:,Role:,Vacancies:
0,Python Engineer,east india securities ltd.,job_description 2 years of experience worki...,"python,hadoop,machinelearning",Posted 5 days ago,2 - 5 yrs,Kolkata,http://www.eisec.com/,IT Software : Software Products & Services,"Petroleum/Oil and Gas/Power, Construction/Cem...",Software Engineer,Any Graduate,NaN,NaN,NaN
1,PYTHON DEVELOPER,DREAMAJAX TECHNOLOGIES,"PYTHON DEVELOPER B.E / B.Tech / MCA / M.Sc., o...","python,django,api,sql,nosql",Posted 6 days ago,4 - 7 yrs,Bengaluru / Bangalore,NaN,IT Software : Software Products & Services,"Telecom, IT-Hardware/Networking",Software Engineer,MCA/ PGDCA\r\n \r\n ...,NaN,NaN,NaN
2,Python Developer,InnOvator Web Solutions Pvt.Ltd.,Job Category: DevelopmentJob Type: Full TimeJo...,"rest,python,django,webdeveloper,mysql,api",Posted 6 days ago,5 - 8 yrs,Mumbai,http://www.innovatorwebsolutions.com,IT Software : Software Products & Services,"IT-Hardware/Networking, Telecom",Software Engineer,Any Graduate,NaN,NaN,NaN


**Step 3 : Explore the data (Shape, Nulls)**

In [4]:
vacancies.shape

(2250, 15)

In [5]:
vacancies[vacancies.columns[vacancies.isnull().any()]].isnull().sum()

Location              71
Website              296
Hiring Location:    2244
Role:               2163
Vacancies:          2238
dtype: int64

Note :  We are able to see that the columns "Hiring Location" , "Role", "Vacancies" contain maximum number of nulls and the above columns are not required for our analysis, we can remove the columns from our dataframe

**Step 4 : Eliminating unrequired columns**

In [6]:
vacancies = vacancies.drop(["Hiring Location:","Role:"],axis = 1)

In [7]:
vacancies.head(3)

,Job Name,Company Name,JD,Skills,Date Posted,YOE,Location,Website,Job Function:,Industry:,Specialization:,Qualification:,Vacancies:
0,Python Engineer,east india securities ltd.,job_description 2 years of experience worki...,"python,hadoop,machinelearning",Posted 5 days ago,2 - 5 yrs,Kolkata,http://www.eisec.com/,IT Software : Software Products & Services,"Petroleum/Oil and Gas/Power, Construction/Cem...",Software Engineer,Any Graduate,NaN
1,PYTHON DEVELOPER,DREAMAJAX TECHNOLOGIES,"PYTHON DEVELOPER B.E / B.Tech / MCA / M.Sc., o...","python,django,api,sql,nosql",Posted 6 days ago,4 - 7 yrs,Bengaluru / Bangalore,NaN,IT Software : Software Products & Services,"Telecom, IT-Hardware/Networking",Software Engineer,MCA/ PGDCA\r\n \r\n ...,NaN
2,Python Developer,InnOvator Web Solutions Pvt.Ltd.,Job Category: DevelopmentJob Type: Full TimeJo...,"rest,python,django,webdeveloper,mysql,api",Posted 6 days ago,5 - 8 yrs,Mumbai,http://www.innovatorwebsolutions.com,IT Software : Software Products & Services,"IT-Hardware/Networking, Telecom",Software Engineer,Any Graduate,NaN


**Step 5 : Eliminating the ":" from the column names**

In [8]:
vacancies.rename(columns = lambda x:x.replace(':',''), inplace = True )

In [9]:
vacancies.head(2)

,Job Name,Company Name,JD,Skills,Date Posted,YOE,Location,Website,Job Function,Industry,Specialization,Qualification,Vacancies
0,Python Engineer,east india securities ltd.,job_description 2 years of experience worki...,"python,hadoop,machinelearning",Posted 5 days ago,2 - 5 yrs,Kolkata,http://www.eisec.com/,IT Software : Software Products & Services,"Petroleum/Oil and Gas/Power, Construction/Cem...",Software Engineer,Any Graduate,NaN
1,PYTHON DEVELOPER,DREAMAJAX TECHNOLOGIES,"PYTHON DEVELOPER B.E / B.Tech / MCA / M.Sc., o...","python,django,api,sql,nosql",Posted 6 days ago,4 - 7 yrs,Bengaluru / Bangalore,NaN,IT Software : Software Products & Services,"Telecom, IT-Hardware/Networking",Software Engineer,MCA/ PGDCA\r\n \r\n ...,NaN


**Step 6 : Standardize the Job names in camel casing**

In [10]:
# create a function to convert a string to camel case
def to_camel(s):
    part = s.split()
    cc = [part[0].capitalize()] + [word.title() for word in part[1:]]
    return '  '.join(cc)

In [11]:
# Applying the function on Job Name
vacancies["Job Name"] = vacancies['Job Name'].apply(to_camel)

In [12]:
#vacancies = vacancies.drop(["Job Name Camel"],axis = 1)

In [13]:
# uppercase company name

vacancies['Company Name'] = vacancies['Company Name'].str.upper()

In [14]:
# lowercase skills

vacancies['Skills'] = vacancies['Skills'].str.lower()

**Step 7 : Check for work from home or work from office and create a separate column**

In [15]:
# Creating a function for WFH and WFO

def work_loc(location):
    if 'Work from Home' in location.lower():
        return 'Work from Home'
    else :
        return 'Work from Office'

In [16]:
#Applying the function to get the work location

vacancies["Work Location"] = vacancies["Date Posted"].apply(work_loc)

**Step 8 : Extract the days before its posted**

In [17]:
# Function to extract number of days

def no_of_days(days):
    pattern = r'(\d+) day(?:s)? ago'
    match = re.search(pattern, days)
    if match:
        return (match.group(1)) + " " + "days"
    elif 'few' in days:
        return "few" + " " +"days" # Assuming 'few' means around 3 days
    elif 'today' in days:
        return "1" + " " +"days" # Assuming 'yesterday' means 1 day ago
    elif 'month' in days:
        return "month" + " " +"ago"  # Assuming 'yesterday' means 1 day ago
    else:
        return None

In [18]:
vacancies['Posted'] = vacancies['Date Posted'].apply(no_of_days)

In [19]:
vacancies.head()

,Job Name,Company Name,JD,Skills,Date Posted,YOE,Location,Website,Job Function,Industry,Specialization,Qualification,Vacancies,Work Location,Posted
0,Python Engineer,EAST INDIA SECURITIES LTD.,job_description 2 years of experience worki...,"python,hadoop,machinelearning",Posted 5 days ago,2 - 5 yrs,Kolkata,http://www.eisec.com/,IT Software : Software Products & Services,"Petroleum/Oil and Gas/Power, Construction/Cem...",Software Engineer,Any Graduate,NaN,Work from Office,5 days
1,Python Developer,DREAMAJAX TECHNOLOGIES,"PYTHON DEVELOPER B.E / B.Tech / MCA / M.Sc., o...","python,django,api,sql,nosql",Posted 6 days ago,4 - 7 yrs,Bengaluru / Bangalore,NaN,IT Software : Software Products & Services,"Telecom, IT-Hardware/Networking",Software Engineer,MCA/ PGDCA\r\n \r\n ...,NaN,Work from Office,6 days
2,Python Developer,INNOVATOR WEB SOLUTIONS PVT.LTD.,Job Category: DevelopmentJob Type: Full TimeJo...,"rest,python,django,webdeveloper,mysql,api",Posted 6 days ago,5 - 8 yrs,Mumbai,http://www.innovatorwebsolutions.com,IT Software : Software Products & Services,"IT-Hardware/Networking, Telecom",Software Engineer,Any Graduate,NaN,Work from Office,6 days
3,Python - Odoo,AXISTECHNOLABS,Responsibilities : We are looking for Freshers...,"python,django,itskills,html5,api,jquery",Posted 6 days ago,0 - 1 yrs,Ahmedabad,http://www.axistechnolabs.com,IT Software : Software Products & Services,"IT-Hardware/Networking, Telecom",Software Engineer,MCA/ PGDCA\r\n \r\n ...,NaN,Work from Office,6 days
4,Python Developer,PEARL GLOBAL SOLUTIONS,"Python Developer Django Job Type : Full time,...","python,database,django,teamplayer,sql",Posted 6 days ago,4 - 7 yrs,Cochin/ Kochi/ Ernakulam,https://pearlglobalsolutions.com/,IT Software : Software Products & Services,"IT-Hardware/Networking, Telecom",Software Engineer,Any Graduate,NaN,Work from Office,6 days


In [20]:
# Drop Date Posted column

vacancies = vacancies.drop(["Date Posted"],axis = 1)

In [21]:
# Initializing sql for testing the changes
pysqldf = lambda q1: sqldf(q1, globals())

In [22]:
q1 = """SELECT * 
       FROM vacancies LIMIT 1;"""

names = pysqldf(q1)
names

,Job Name,Company Name,JD,Skills,YOE,Location,Website,Job Function,Industry,Specialization,Qualification,Vacancies,Work Location,Posted
0,Python Engineer,EAST INDIA SECURITIES LTD.,job_description 2 years of experience worki...,"python,hadoop,machinelearning",2 - 5 yrs,Kolkata,http://www.eisec.com/,IT Software : Software Products & Services,"Petroleum/Oil and Gas/Power, Construction/Cem...",Software Engineer,Any Graduate,None,Work from Office,5 days


**Step 9 : Min years of experience required**

In [23]:
# Getting the min years of experience required for job

pattern = r"(\d+)\s*-\s*(\d+)\s*(?:years|yrs)"

def extract_min_experience(exp_str):
    match = re.search(pattern, exp_str)
    if match:
        return int(match.group(1))
    else:
        return None

In [24]:
# Applying the function to get a new column with min years of exp

vacancies['min_years_experience'] = vacancies['YOE'].apply(extract_min_experience)

In [25]:
vacancies.head(2)

,Job Name,Company Name,JD,Skills,YOE,Location,Website,Job Function,Industry,Specialization,Qualification,Vacancies,Work Location,Posted,min_years_experience
0,Python Engineer,EAST INDIA SECURITIES LTD.,job_description 2 years of experience worki...,"python,hadoop,machinelearning",2 - 5 yrs,Kolkata,http://www.eisec.com/,IT Software : Software Products & Services,"Petroleum/Oil and Gas/Power, Construction/Cem...",Software Engineer,Any Graduate,NaN,Work from Office,5 days,2
1,Python Developer,DREAMAJAX TECHNOLOGIES,"PYTHON DEVELOPER B.E / B.Tech / MCA / M.Sc., o...","python,django,api,sql,nosql",4 - 7 yrs,Bengaluru / Bangalore,NaN,IT Software : Software Products & Services,"Telecom, IT-Hardware/Networking",Software Engineer,MCA/ PGDCA\r\n \r\n ...,NaN,Work from Office,6 days,4


## Exploratory Data Analysis

**1 : Location with most & least jobs**

In [26]:
# get the count of job names by location

location_count = vacancies.groupby('Location')['Job Name'].count().reset_index()

In [27]:
# Location with most Jobs

max_location = location_count.loc[location_count['Job Name'].idxmax()]

In [28]:
# Location with min Jobs

min_location = location_count.loc[location_count['Job Name'].idxmin()]

In [29]:
# Print max & min values

print(f"Location with the most job names: {max_location['Location']} ({max_location['Job Name']} job names)")
print(f"Location with the least job names: {min_location['Location']} ({min_location['Job Name']} job names)")

Location with the most job names: Bengaluru / Bangalore (412 job names)
Location with the least job names: Ahmedabad,  Bengaluru / Bangalore (1 job names)


**2. Industry with max & min jobs**

In [30]:
# get the count of job names by industry

industry_count = vacancies.groupby('Industry')['Job Name'].count().reset_index()

In [31]:
# Industry with most jobs

max_industries = industry_count.loc[industry_count['Job Name'].idxmax()]

In [32]:
# Industry with min jobs

min_industries = industry_count.loc[industry_count['Job Name'].idxmin()]

In [33]:
# Print the Industry with Max and Min jobs

print(f"Industry with the most job names: {max_industries['Industry']} ({max_industries['Job Name']} job names)")
print(f"Industry with the least job names: {min_industries['Industry']} ({min_industries['Job Name']} job names)")

Industry with the most job names: IT-Hardware/Networking,  Telecom (1072 job names)
Industry with the least job names: Accounting-Tax/Consulting,  Advertising/PR/Event Management (1 job names)


**3. Specilization with most jobs**

In [34]:
# get the count of jobs by specilization

specialization_count = vacancies.groupby('Specialization')['Job Name'].count().reset_index()

In [35]:
# Industry with most jobs

max_specializations = specialization_count.loc[specialization_count['Job Name'].idxmax()]

In [36]:
# Print the Specilization with Max jobs

print(f"Specilization with the most job names: {max_specializations['Specialization']} ({max_specializations['Job Name']} job names)")

Specilization with the most job names: Software Engineer (1654 job names)


**4. Most required skills across various Industries**

In [37]:
Temp_df = vacancies

In [38]:
# Splitting Skills into Individual Rows

skills = Temp_df['Skills'].str.split(',',expand=True).stack().reset_index(level=1, drop=True).rename('Skill')
skills = Temp_df.drop('Skills', axis = 1).join(skills).reset_index(drop=True)

In [39]:
# Get the most required skills

skill_count = skills['Skill'].value_counts()
print(skill_count)

Skill
javascript         838
css                658
html               648
java               449
jquery             400
                  ... 
teamleader           1
tallyaccounting      1
e                    1
emailwriting         1
dispatch             1
Name: count, Length: 1724, dtype: int64


In [40]:
# checking for python

python_requirements = (skills['Skill'] == 'python').sum()
print(python_requirements)

373


**5. Company Requirement Trend**

In [42]:
# get the count of job names by company

company_count = vacancies.groupby('Company Name')['Job Name'].count().reset_index()

In [43]:
# companies with max and least jobs

max_req_comp = company_count.loc[company_count['Job Name'].idxmax()]
min_req_comp = company_count.loc[company_count['Job Name'].idxmin()]

print(f"Companies with the most job names: {max_req_comp['Company Name']} ({max_req_comp['Job Name']} job names)")
print(f"Companies with the least job names: {min_req_comp['Company Name']} ({min_req_comp['Job Name']} job names)")

Companies with the most job names: HYREFOX CONSULTANTS (67 job names)
Companies with the least job names: 2ASQUARE INFOTECH STUDIOS (1 job names)


**6. Recent Job Buckets**

In [45]:
# Getting the recent job buckets using posted column

job_bucket = vacancies['Posted'].value_counts().sort_index()

In [46]:
print(job_bucket)

Posted
1 days         30
2 days         69
3 days         50
4 days         64
5 days         83
6 days        676
few days     1160
month ago     118
Name: count, dtype: int64
